*   실행환경: 구글 코랩
*   작성일: 230929
*   작성자: 이혜원



### 라이브러리 설치, 불러오기

In [ ]:
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [20]:
#라이브러리 불러오기
import pandas as pd
import re
from konlpy.tag import Mecab

### 데이터 불러오기

In [2]:
song_df = pd.read_csv('가사.csv')
rank_df = pd.read_csv('순위.csv')
info_df = pd.read_csv('앨범정보.csv')

### 사용자 함수 정의하기

In [37]:
# 가사 전처리
def preprocess(sentences):
    lst = []
    for sentence in sentences:
        # 한국어, 공백 빼고 제거하기'
        sentence = re.sub('[^ㄱ-ㅎ가-힣\s]+', '', sentence)

        lst.append(sentence)
    return lst

In [45]:
# 형태소 분석기 mecab을 사용한 토큰화 작업 (다소 시간 소요)
def tokenize(sentences):
    mecab = Mecab()
    tokenized_data = []
    for sentence in sentences:
        temp_X = mecab.morphs(sentence)
        temp_X = [word for word in temp_X]
        tokenized_data.append(temp_X)
    return tokenized_data

### 전처리

In [12]:
lyrics_lst = song_df['Lyrics']

In [97]:
tmp = lyrics_lst[:2]
tmp

0    어린 햇살 아래서뛰어놀곤 했었던가쁜 숨결굽이진 골목 지나길을 따라가보면같은 기억어른...
1    꽃이 언제 피는지 그딴 게 뭐가 중요한데날씨가 언제 풀리는지 그딴 거 알면 뭐 할 ...
Name: Lyrics, dtype: object

In [ ]:
sentences = preprocess(tmp)
tokenize(sentences)

In [67]:
# song_id로 rank_df 행 찾기
def find_rank_song(song_id):
    return rank_df[rank_df['song_id'] ==  song_id]

# album_id로 rank_df 행 찾기
def find_rank_album(album_id):
    return rank_df[rank_df['album_id'] ==  album_id]

# album_id로 info_df 행 찾기
def find_info_id(album_id):
    return info_df[info_df['AlbumID'] == album_id]

In [73]:
album_id = find_rank_song(8124417)['album_id'].values[0]
#album_id
find_info_id(album_id)

,Unnamed: 0,AlbumID,Release_date,Genre,Publisher,Agency
576,66,2676884,2016.04.01,"인디음악, 포크/블루스",포크라노스,매직스트로베리사운드


### 장르 unique 개수 구하기

In [94]:
unique_lst = info_df['Genre'].unique()
print(len(unique_lst))
unique_lst[:10]

89


array(['발라드', '록/메탈', '랩/힙합', '댄스, 발라드', 'R&B/Soul', '발라드, 댄스', '댄스',
       '댄스, 랩/힙합, 록/메탈', '발라드, 록/메탈, 포크/블루스', '인디음악, 포크/블루스'],
      dtype=object)

In [84]:
# 한 행의 장르가 '댄스, 랩/힙합, 록/메탈' 로 콤마로 여러개가 표시되어 있는 경우가 있음
split_lst = [val.split(', ') for val in unique_lst]

In [101]:
sum_lst = sum(split_lst, [])  # 리스트 내 리스트 합치기
genre_lst = list(set(sum_lst))  # 중복 제거
print(len(genre_lst))
genre_lst

14


['국내드라마',
 '크로스오버',
 '재즈',
 '포크/블루스',
 '랩/힙합',
 '애시드/퓨전/팝',
 '일렉트로니카',
 '록/메탈',
 'R&B/Soul',
 '인디음악',
 '성인가요/트로트',
 '클래식',
 '댄스',
 '발라드']

In [ ]:
# 한 노래의 장르가 여러개일 경우 어떻게 처리하지?
# 예측할 때 하나라도 맞추면 맞았다고 해야하는지?
# 다중 레이블 분류 찾아보기